In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import hummingbird.ml

Importing plotly failed. Interactive plots will not work.


#### Create and fit the model

In [2]:
# Read the data and train a Prophet model
DATA = "../tests/resources/example_wp_log_peyton_manning.csv"
df = pd.read_csv(DATA)

model = Prophet()
model.fit(df)
        

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [3]:
# Convert into PyTorch using Hummingbird
hb_model = hummingbird.ml.convert(model, "torch")

In [4]:
# Generate future dataset.
future = model.make_future_dataframe(periods=365)

In [5]:
# Trend prediction with prophet
prophet_trend = model.predict(future)["trend"]

In [6]:
# Trend prediction with Hummingbird
hb_trend = hb_model.predict(future)

In [7]:
# Check that the predictions match.
np.testing.assert_allclose(prophet_trend, hb_trend, rtol=1e-06, atol=1e-06)

In [8]:
%%timeit
prophet_trend = model.predict(future)

6.26 ms ± 61.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
hb_trend = hb_model.predict(future)

5.77 ms ± 70.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
# Generate onnx model. ONNX model requires inputs in numpy format.
future_np = (future.values - np.datetime64("1970-01-01T00:00:00.000000000")).astype(np.int64) / 1000000000
hb_onnx_model = hummingbird.ml.convert(model, "onnx", future_np)

/home/hummingbird/anaconda/lib/python3.7/site-packages/torch/tensor.py:590: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  'incorrect results).', category=RuntimeWarning)


In [11]:
%%timeit
hb_trend = hb_onnx_model.predict(future_np)

10.7 ms ± 72.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
# Test on GPU
hb_model.to('cuda')

In [13]:
%%timeit
hb_trend = hb_model.predict(future)

11.7 ms ± 72.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
